# Описание проекта

Сетевой супермаркет «Хлеб-Соль» внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:
Анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
Контролировать добросовестность кассиров при продаже алкоголя.
Постройте модель, которая по фотографии определит приблизительный возраст человека. В вашем распоряжении набор фотографий людей с указанием возраста.


# Инструкция по выполнению

Проведите исследовательский анализ набора фотографий.
Подготовьте данные к обучению.
Обучите нейронную сеть и рассчитайте её качество.

# Описание данных

Данные взяты с сайта ChaLearn Looking at People. Они находятся в папке /datasets/faces/.

# 1. исследовательский анализ данных

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
df_train = pd.read_csv('appa-real-release/gt_train.csv')
df_train.head()

,file_name,real_age,apparent_age,worker_age,worker_gender
0,000000.jpg,4,5,36,male
1,000000.jpg,4,4,38,female
2,000000.jpg,4,7,52,female
3,000000.jpg,4,5,50,male
4,000000.jpg,4,4,52,female


In [18]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory='appa-real-release/train',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345)

Found 133935 validated image filenames.


In [20]:
print(labels.info())   
labels['real_age'].hist(bins=100)
plt.title('распределение покупателей по возрасту')
plt.show()


features, target = next(train_gen_flow)

fig = plt.figure(figsize=(10,10))
for i in range(12):
    fig.add_subplot(3, 4, i+1)
    plt.imshow(features[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

NameError: name 'labels' is not defined

В выборке 7591 фотография.
Больше всего людей в возрасте 20-30 лет.
Присутствуют пики на круглых числах (30,35,40,50,60,65,70) - возможно возраст определялся не точно: экспертом или в результате анкетирования. Результат обучения будет зависеть от такого неравномерного распределения и модель будет склонна округлять возраст.

# 2. Обучение модели

запустить нижеприведенную модель в GPU

from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_train(path):
    train_datagen = ImageDataGenerator( validation_split=0.25, rescale=1. /255 ) 

    train_datagen_flow = train_datagen.flow_from_directory(
        path, 
        target_size=(224, 224),
        batch_size=16,
        class_mode='sparse',
        subset='training',
        seed=12345)

    return train_datagen_flow

def load_test(path):
    test_datagen = ImageDataGenerator( validation_split=0.25, rescale=1. /255 ) 

    test_datagen_flow = test_datagen.flow_from_directory(
        path, 
        target_size=(224, 224),
        batch_size=16,
        class_mode='sparse',
        subset='validation', # !! было 'training'
        seed=12345)

    return test_datagen_flow

def create_model(input_shape):
    model = ResNet50(input_shape=None, classes=1000, include_top=True, weights='imagenet')

    backbone = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1))

    optimizer = Adam(lr=0.0001)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

    return model


def train_model(model, train_data, test_data, batch_size=16, epochs=5, steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data,
              validation_data=test_data,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

    return model

Train for 356 steps, validate for 356 steps
2020-05-05 07:16:07.131617: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-05-05 07:16:08.510290: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
356/356 - 91s - loss: 0.1357 - mae: 0.2659 - val_loss: 0.0069 - val_mae: 0.0664
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
356/356 - 31s - loss: 0.0069 - mae: 0.0664